In [1]:
import os
os.environ['HUGGINGFACE_CACHE'] = '/share/huggingface/'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
import json
# sys.path.append(os.getcwd())
sys.path.append('../')
from easyeditor import LoRAHyperParams
from easyeditor import BaseEditor
# from easyeditor.models.ike import encode_ike_facts
# from sentence_transformers import SentenceTransformer
from easyeditor import KnowEditDataset

In [2]:
data_dir = '../dataset/ccks2024_know_edit/ZsRE-test-all.json'
train_data_path = None
ds_size, data_type, = 326, 'zsre'
hparams_dir = '../hparams/LoRA/Meta-Llama-3-8B-Instruct.yaml'
metrics_save_dir = './EasyEditCache/metrics'

In [5]:
datas = KnowEditDataset(data_dir,size=ds_size)
if data_type == 'counterfact' or data_type == 'recent' or data_type == 'zsre':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    portability_r =[data['portability_r'] for data in datas]
    portability_s =[data['portability_s'] for data in datas]
    portability_l =[data['portability_l'] for data in datas]

    portability_reasoning_prompts=[]
    portability_reasoning_ans=[]
    portability_Logical_Generalization_prompts=[]
    portability_Logical_Generalization_ans=[]
    portability_Subject_Aliasing_prompts=[]
    portability_Subject_Aliasing_ans=[]
    
    portability_data = [portability_r,portability_s,portability_l]
    portability_prompts = [portability_reasoning_prompts,portability_Subject_Aliasing_prompts,portability_Logical_Generalization_prompts]
    portability_answers = [portability_reasoning_ans,portability_Subject_Aliasing_ans,portability_Logical_Generalization_ans]
    for data, portable_prompts, portable_answers in zip(portability_data,portability_prompts,portability_answers):
        for item in data:
            if item is None:
                portable_prompts.append(None)
                portable_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                portable_prompts.append(temp_prompts)
                portable_answers.append(temp_answers)
    assert len(prompts) == len(portability_reasoning_prompts) == len(portability_Logical_Generalization_prompts) == len(portability_Subject_Aliasing_prompts)
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    locality_Forgetfulness_prompts=[]        
    locality_Forgetfulness_ans=[]
    
    locality_data = [locality_rs, locality_f]
    locality_prompts = [locality_Relation_Specificity_prompts,locality_Forgetfulness_prompts]
    locality_answers = [locality_Relation_Specificity_ans,locality_Forgetfulness_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts) == len(locality_Forgetfulness_prompts)
    locality_inputs = {}
    portability_inputs = {}
    
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        },
        'Forgetfulness':{
            'prompt':locality_Forgetfulness_prompts,
            'ground_truth':locality_Forgetfulness_ans
        }
    }
    portability_inputs = {
        'Subject_Aliasing':{
            'prompt': portability_Subject_Aliasing_prompts,
            'ground_truth': portability_Subject_Aliasing_ans
        },
        'reasoning':{
            'prompt': portability_reasoning_prompts,
            'ground_truth': portability_reasoning_ans           
        },
        'Logical_Generalization':{
            'prompt': portability_Logical_Generalization_prompts,
            'ground_truth': portability_Logical_Generalization_ans           
        }
    }
if data_type == 'wikibio':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    
    locality_data = [locality_rs]
    locality_prompts = [locality_Relation_Specificity_prompts]
    locality_answers = [locality_Relation_Specificity_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts)
    portability_inputs = None
    locality_inputs = {}
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        }
    }

hparams = LoRAHyperParams.from_hparams(hparams_dir)
pre_file = f"../pre_edit/{hparams.model_name.split('/')[-1]}_{data_type}_pre_edit.json"
if pre_file is not None and os.path.exists(pre_file):
    pre_edit = json.load(open(pre_file,'r'))[:ds_size]
    assert len(pre_edit) == len(prompts)
else:
    pre_edit = None

train_ds = None

In [7]:
editor = BaseEditor.from_hparams(hparams)

2024-07-22 11:17:32,477 - easyeditor.editors.editor - INFO - Instantiating model
07/22/2024 11:17:32 - INFO - easyeditor.editors.editor -   Instantiating model


Using Huggingface cache: /share/huggingface/Meta-Llama-3-8B-Instruct


07/22/2024 11:17:34 - INFO - accelerate.utils.modeling -   We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
with open('./0-326-Meta-Llama-3-8B-Instruct-zsre-knb_orgin_dict.json', 'r') as f:
    knb_dict = json.load(f)

In [9]:
# 单条数据编辑
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    target_new=target_new,
    subject=subjects,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    train_ds=train_ds,
    keep_original_weight=True,
    pre_file=pre_file,
    pre_edit = pre_edit,
    test_generation=True,
    knb_dict = knb_dict,
)

  0%|          | 0/326 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Executing LoRA algo for: [Which family does Epaspidoceras belong to?] -> [Noctuidae]
Using device: cuda:0
Epoch: 0
Batch loss 5.207841396331787
Total loss 5.207841396331787
Epoch: 1
Batch loss 1.5462923049926758
Total loss 1.5462923049926758
Epoch: 2
Batch loss 0.8799525499343872
Total loss 0.8799525499343872
Epoch: 3
Batch loss 2.377507209777832
Total loss 2.377507209777832
Epoch: 4
Batch loss 9.015673637390137
Total loss 9.015673637390137
Epoch: 5
Batch loss 13.061845779418945
Total loss 13.061845779418945
Epoch: 6
Batch loss 5.843803882598877
Total loss 5.843803882598877
Epoch: 7
Batch loss 12.782012939453125
Total loss 12.782012939453125
Epoch: 8
Batch loss 9.755782127380371
Total loss 9.755782127380371
Epoch: 9
Batch loss 8.063555717468262
Total loss 8.063555717468262
Epoch: 10
Batch loss 4.7052459716796875
Total loss 4.7052459716796875
Epoch: 11
Batch loss 4.494955539703369
Total loss 4.49

2024-07-22 11:57:14,377 - easyeditor.editors.editor - INFO - 0 editing: Which family does Epaspidoceras belong to? -> Noctuidae  

 {'pre': {'rewrite_acc': [0.25], 'portability': {'reasoning_acc': [0.25]}, 'fluency': {'ngram_entropy': 5.972940386324494}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Which family does Epaspidoceras belong to?', 'target_new': 'Noctuidae', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['What is the common name for the family Epaspidoceras belongs to?'], 'ground_truth': ['Owlet moths']}}, 'locality': {'Relation_Specificity': {'prompt': ['The taxon rank of Epaspidoceras is', 'Epaspidoceras taxon rank'], 'ground_truth': ['genus', 'genus']}}, 'subject': 'Epaspidoceras'}, 'post': {'rewrite_acc': [0.5], 'locality': {'Relation_Specificity_acc': [0.0, 0.0]}, 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 4.414688821398273}}}
07/22/2024 11:57:14 - INFO - easyeditor.editors.editor -   0 editing: Which family does 

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Executing LoRA algo for: [What species is ZIC3 specific to?] -> [male]
Using device: cuda:0
Epoch: 0
Batch loss 15.99152660369873
Total loss 15.99152660369873
Epoch: 1
Batch loss 3.4902329444885254
Total loss 3.4902329444885254
Epoch: 2
Batch loss 0.017483990639448166
Total loss 0.017483990639448166
Epoch: 3
Batch loss 0.01864689588546753
Total loss 0.01864689588546753
Epoch: 4
Batch loss 5.960462772236497e-07
Total loss 5.960462772236497e-07
Epoch: 5
Batch loss 7.152555099310121e-07
Total loss 7.152555099310121e-07
Epoch: 6
Batch loss 1.1920928244535389e-07
Total loss 1.1920928244535389e-07
Epoch: 7
Batch loss 60.850685119628906
Total loss 60.850685119628906
Epoch: 8
Batch loss 9.83560562133789
Total loss 9.83560562133789
Epoch: 9
Batch loss 51.61858367919922
Total loss 51.61858367919922
Epoch: 10
Batch loss 15.453568458557129
Total loss 15.453568458557129
Epoch: 11
Batch loss 0.007349951192736

2024-07-22 11:57:43,633 - easyeditor.editors.editor - INFO - 1 editing: What species is ZIC3 specific to? -> male  

 {'pre': {'rewrite_acc': [0.0], 'portability': {'Subject_Aliasing_acc': [0.0]}, 'fluency': {'ngram_entropy': 6.276496530413857}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'What species is ZIC3 specific to?', 'target_new': 'male', 'ground_truth': '<|endoftext|>', 'portability': {'Subject_Aliasing': {'prompt': ['What is the function of Zic family member 3?'], 'ground_truth': ['male']}}, 'locality': {'Relation_Specificity': {'prompt': ['The strand orientation of ZIC3 is', 'ZIC3 strand orientation'], 'ground_truth': ['forward strand', 'forward strand']}}, 'subject': 'ZIC3'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [0.0, 0.0]}, 'portability': {'Subject_Aliasing_acc': [1.0]}, 'fluency': {'ngram_entropy': 0.5847273945620163}}}
07/22/2024 11:57:43 - INFO - easyeditor.editors.editor -   1 editing: What species is ZIC3 specific to? -> male  

 

...

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Executing LoRA algo for: [Who created Holmenkollen Chapel?] -> [Norwegian Institute of Technology]
Using device: cuda:0
Epoch: 0
Batch loss 4.0764851570129395
Total loss 4.0764851570129395
Epoch: 1
Batch loss 0.9320060014724731
Total loss 0.9320060014724731
Epoch: 2
Batch loss 0.00969909131526947
Total loss 0.00969909131526947
Epoch: 3
Batch loss 0.1298617124557495
Total loss 0.1298617124557495
Epoch: 4
Batch loss 21.29031753540039
Total loss 21.29031753540039
Epoch: 5
Batch loss 11.53962516784668
Total loss 11.53962516784668
Epoch: 6
Batch loss 7.326054096221924
Total loss 7.326054096221924
Epoch: 7
Batch loss 8.838090896606445
Total loss 8.838090896606445
Epoch: 8
Batch loss 8.117365837097168
Total loss 8.117365837097168
Epoch: 9
Batch loss 8.328568458557129
Total loss 8.328568458557129
Epoch: 10
Batch loss 5.734803199768066
Total loss 5.734803199768066
Epoch: 11
Batch loss 4.534731388092041
T

2024-07-22 13:10:58,984 - easyeditor.editors.editor - INFO - 196 editing: Who created Holmenkollen Chapel? -> Norwegian Institute of Technology  

 {'pre': {'rewrite_acc': [0.25], 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 5.260047612937635}}, 'case_id': 196, 'requested_rewrite': {'prompt': 'Who created Holmenkollen Chapel?', 'target_new': 'Norwegian Institute of Technology', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['In which city is the institution responsible for creating Holmenkollen Chapel located?'], 'ground_truth': ['Trondheim']}}, 'locality': {'Relation_Specificity': {'prompt': ['The architectural style of Holmenkollen Chapel is', 'Holmenkollen Chapel architectural style'], 'ground_truth': ['Art Nouveau', 'Art Nouveau']}}, 'subject': 'Holmenkollen Chapel'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [0.0, 0.0]}, 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 4.48075978

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Executing LoRA algo for: [What city is the band Mini Mansions from?] -> [Manchester]
Using device: cuda:0
Epoch: 0
Batch loss 9.274489402770996
Total loss 9.274489402770996
Epoch: 1
Batch loss 0.18232765793800354
Total loss 0.18232765793800354
Epoch: 2
Batch loss 0.02416316047310829
Total loss 0.02416316047310829
Epoch: 3
Batch loss 9.60780744208023e-05
Total loss 9.60780744208023e-05
Epoch: 4
Batch loss 4.541770613286644e-05
Total loss 4.541770613286644e-05
Epoch: 5
Batch loss 3.540453326422721e-05
Total loss 3.540453326422721e-05
Epoch: 6
Batch loss 4.541770613286644e-05
Total loss 4.541770613286644e-05
Epoch: 7
Batch loss 3.540453326422721e-05
Total loss 3.540453326422721e-05
Epoch: 8
Batch loss 5.829164365422912e-05
Total loss 5.829164365422912e-05
Epoch: 9
Batch loss 5.829164365422912e-05
Total loss 5.829164365422912e-05
Epoch: 10
Batch loss 4.541770613286644e-05
Total loss 4.54177061328664

2024-07-22 13:11:19,584 - easyeditor.editors.editor - INFO - 197 editing: What city is the band Mini Mansions from? -> Manchester  

 {'pre': {'rewrite_acc': [0.0], 'portability': {'reasoning_acc': [0.6666666666666666]}, 'fluency': {'ngram_entropy': 6.309178949240535}}, 'case_id': 197, 'requested_rewrite': {'prompt': 'What city is the band Mini Mansions from?', 'target_new': 'Manchester', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['In which region of England did the band Mini Mansions originate?'], 'ground_truth': ['North West England']}}, 'locality': {'Relation_Specificity': {'prompt': ['The record label of Mini Mansions is', 'Mini Mansions record label'], 'ground_truth': ['Capitol Records', 'Capitol Records']}}, 'subject': 'Mini Mansions'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [0.0, 0.0]}, 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 0.7439666152167319}}}
07/22/2024 13:11:19 - INFO - easyeditor

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Executing LoRA algo for: [The inventor of Penrose stairs was whom?] -> [Richard Penrose]
Using device: cuda:0
Epoch: 0
Batch loss 3.5880775451660156
Total loss 3.5880775451660156
Epoch: 1
Batch loss 0.06731399148702621
Total loss 0.06731399148702621
Epoch: 2
Batch loss 0.08691445738077164
Total loss 0.08691445738077164
Epoch: 3
Batch loss 8.03968334197998
Total loss 8.03968334197998
Epoch: 4
Batch loss 3.39133620262146
Total loss 3.39133620262146
Epoch: 5
Batch loss 4.126573085784912
Total loss 4.126573085784912
Epoch: 6
Batch loss 2.9018983840942383
Total loss 2.9018983840942383
Epoch: 7
Batch loss 2.7434442043304443
Total loss 2.7434442043304443
Epoch: 8
Batch loss 1.9295202493667603
Total loss 1.9295202493667603
Epoch: 9
Batch loss 1.9390058517456055
Total loss 1.9390058517456055
Epoch: 10
Batch loss 1.4052801132202148
Total loss 1.4052801132202148
Epoch: 11
Batch loss 1.6815742254257202
Tota

In [ ]:
if not os.path.exists(metrics_save_dir):
    os.makedirs(metrics_save_dir)
json.dump(metrics, open(os.path.join(metrics_save_dir, f'KNB_LoRA_{data_type}_{ds_size}_{hparams_dir.split("/")[-1]}_results.json'), 'w'), indent=4)